In [1]:
f_path = '/home/u/u241308/script_python/gb_functions'
import sys
sys.path.insert(1, f_path)
import gb_hwdetect_optimized as gb_hw

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import xarray as xr
import copy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
from math import sin, cos, sqrt, atan2, radians

In [2]:
#reference time
ref_min = 1985
ref_max = 2014

# Consecutive days for heatwave detection
c_days = 6

# Threshold percentile
percentile = 90 #95

#Definition type: daily moving threshold (mov_day) or JA moving threshold (mov_JA)
def_type = 'fix_day' #'mov_JA'

In [3]:
def run_hw_detect():    
    #import data
    path = '/work/uo1075/u241308/reanalysis/eobs/eobs_hw_projections/'
    file = 'tasmax_1950_2022_GEgrid2_anomaly_detrended_multi_model.nc'

    with xr.open_dataset(path+file) as data1:
        #data1 = data1.rename({'__xarray_dataarray_variable__':'tx'})
        data1 = data1.tx
        
        #Get rid off 29th of februarys
        data1 = data1.sel(time=~((data1.time.dt.month == 2) & (data1.time.dt.day == 29)))
        
    #load threshold
    path = '/work/uo1075/u241308/data_python/HW_extension/eobs/'
    file = 'threshold_%s'%percentile+ '_fix_day_eobs_ref_%i'%ref_min + '-%i_anomaly_detrended_multi_model.nc'%ref_max  
    
    with xr.open_dataset(path+file) as threshold:
        threshold = threshold.tx
        
        #select one year (all years are the same in fix_day)
        threshold = threshold[(threshold.time.dt.year==threshold.time.dt.year.min())].values
        
    #expand threhsold to dimensions of data1
    year_number = np.unique(data1.time.dt.year).shape[0]
    out_data = copy.deepcopy(data1)
    out_data[:,:,:] = np.tile(threshold,(year_number,1,1))
        
    #detect hws
    heat_wave_day_withtime1 = gb_hw.gb_hwdetect(data1,out_data,c_days)
    out_data[:,:,:] = heat_wave_day_withtime1
            
    return out_data

In [4]:
hw_detect = run_hw_detect()

#save data
path = '/work/uo1075/u241308/data_python/HW_extension/eobs/'
file = 'hw_detect_%id'%c_days +'_%s'%percentile+ '_' + def_type + '_eobs_ref_%i'%ref_min + '-%i_anomaly_detrended_multi_model.nc'%ref_max    
hw_detect.to_netcdf(path+file)

In [5]:
print('code done')

code done
